In [ ]:
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from src.consts import *
from src.validation import validate_jumpstart_cube, display_validate_results
from src.coherence import analyze_deck_theme_coherence_enhanced
from src.improve import apply_swap

load_dotenv(override=True)
openai = OpenAI()
anthropic = Anthropic() 

# Load the data files
oracle_df = pd.read_csv('ThePauperCube_oracle_with_pt.csv')
cube_df = pd.read_csv('JumpstartCube_ThePauperCube_ULTIMATE_Final_v2.csv')

In [5]:
# Debug: Check the data structure
print("=== DATA STRUCTURE DEBUG ===")
print(f"Cube_df shape: {cube_df.shape}")
print(f"Cube_df columns: {list(cube_df.columns)}")
print("\nFirst few rows of cube_df:")
print(cube_df.head())
print(f"\nOracle_df shape: {oracle_df.shape}")
print(f"Oracle_df columns: {list(oracle_df.columns)}")

=== DATA STRUCTURE DEBUG ===
Cube_df shape: (390, 11)
Cube_df columns: ['Name', 'Set', 'Collector Number', 'Rarity', 'Color Identity', 'Type', 'Mana Cost', 'CMC', 'Power', 'Toughness', 'Tags']

First few rows of cube_df:
                    Name    Set  Collector Number  Rarity Color Identity  \
0          Archaeomancer  Mixed               NaN  common              U   
1  Behold the Multiverse  Mixed               NaN  common              U   
2             Ephemerate  Mixed               NaN  common              W   
3         Lyev Skyknight  Mixed               NaN  common             WU   
4        Lórien Revealed  Mixed               NaN  common              U   

                      Type  Mana Cost  CMC  Power  Toughness  \
0  Creature - Human Wizard        NaN    4    NaN        NaN   
1                  Instant        NaN    4    NaN        NaN   
2                  Instant        NaN    1    NaN        NaN   
3  Creature - Human Knight        NaN    3    NaN        NaN   
4 

# Optimise

In [7]:
# You can also analyze a specific deck by name
# Example: Analyze the "Green Big Creatures" deck

import importlib
import src.process
importlib.reload(src.process)

from src.process import optimize_deck_coherence, clear_swap_history

# Clear any previous swap history to prevent issues with oscillation
clear_swap_history()

coherence = analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)
total_coherence = sum(result['overall_coherence'] for result in coherence.values())

improvement = True
iteration = 0
max_iterations = 20  # Prevent infinite loops

while improvement and iteration < max_iterations:
    prev_total_coherence = total_coherence
    cube_df = optimize_deck_coherence(cube_df=cube_df, oracle_df=oracle_df)
    coherence = analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)
    total_coherence = sum(result['overall_coherence'] for result in coherence.values())
    improvement = total_coherence > prev_total_coherence
    iteration += 1
    print(f"Iteration {iteration}: Total coherence: {total_coherence:.2f} (improvement: {total_coherence - prev_total_coherence:.2f})")
    
    if not improvement:
        print("No further improvements found - optimization complete!")
    elif iteration >= max_iterations:
        print("Maximum iterations reached - stopping optimization")

Swap history cleared


Analyzing deck: Blue Flyers

Current coherence: 0.3

Expected themes: Unknown

Deck colors: U

Found 0 candidate cards to consider

❌ **Error:** Not enough candidate cards found. Need 2, found 0

No swaps available for Blue Flyers: Not enough candidate cards found. Need 2, found 0
Iteration 1: Total coherence: 214.19 (improvement: 0.00)
No further improvements found - optimization complete!


# Save to file

In [ ]:
from src.export import export_cube_to_csv

export_cube_to_csv(cube_df, oracle_df, 'JumpstartCube_ThePauperCube_ULTIMATE_Final_v2.csv')

Exporting cube to JumpstartCube_ThePauperCube_ULTIMATE_Final_v4.csv...
✅ Successfully exported 390 cards to JumpstartCube_ThePauperCube_ULTIMATE_Final_v4.csv

📊 Export Summary:
Total cards: 390
Number of decks: 30

Deck breakdown:
  Azorius Evasion/Flying: 13 cards
  Black Aggro: 13 cards
  White Equipment: 13 cards
  White Control: 13 cards
  White Aggro: 13 cards
  Simic Control: 13 cards
  Selesnya Control: 13 cards
  Red Artifacts: 13 cards
  Red Burn: 13 cards
  Rakdos Burn/Damage: 13 cards
  ... and 20 more decks


'JumpstartCube_ThePauperCube_ULTIMATE_Final_v4.csv'

# Analysis

In [ ]:
# from src.coherence import display_coherence_analysis_enhanced


# display_coherence_analysis_enhanced(analyze_deck_theme_coherence_enhanced(cube_df, oracle_df))